
# <center>手写数字识别案例</center>

## 使用CNN手写数字识别

  * 黑白图片n✖m✖1
![](images/cnn_mnist.png)

### 1.1、引入各类库

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

### 1.2、对模型输入进行设置，并加载数据

In [ ]:
batch_size = 128
num_classes = 10
epochs = 2

# 图片像素点
img_rows, img_cols = 28, 28

# 加载数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# 查看数据形状
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
# 查看数据形状
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
# 查看数据值及类型
x_train[0]

In [ ]:
# 查看数据值及类型
x_test[0].shape

这里用卷积神经网络来对图像做特征处理，一般来说，输入到网络的图像格式有以下两种：
1. channels_first (batch_size,channels,width,height)
1. channels_last  (batch_size,width,height,channels)

这里channels指的是通道数，灰度图是单通道channels=1，彩色图是三通道channels=3，需要注意的是，即使图像是单通道的，输入数据的维度依然是4维。反观我们的mnist图像数据，只有三维，所以我们要手动把channels这个维度加上。由于Keras使用不同后端的时候，数据格式不一样，所以要分情况进行维度增加

值得注意的是，reshape函数第一个参数为-1，意思为保持当前维度不变

![](images/pic.png)

In [ ]:
# 判断图像通道数
print(K.image_data_format())

In [ ]:
# 转换成图像数据
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [ ]:
# 查看数据形状
print(x_train.shape, x_test.shape)

### 1.3、数据归一化

In [ ]:
# 查看数据值及类型
x_train[0]

In [ ]:
# 查看数据值及类型
x_test[0]

In [ ]:
# 转换为浮点型
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
# 查看数据值及类型
x_train[0]

In [ ]:
# 数据归一化
x_train = x_train/255
x_test = x_test/255

In [ ]:
# 查看数据值及类型
x_train[0]

In [ ]:
y_train[:5]

In [ ]:
# y值one-hot编码
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train,10)
y_test = np_utils.to_categorical(y_test,10)

In [ ]:
y_train[:5]

In [ ]:
# 查看数据处理完后的数据及形状
x_train.shape

In [ ]:
y_train.shape

### 1.4、预览数据

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 6))
for i in range(0, num_classes):
    plt.subplot(2, 5, i+1)
    x_selected = x_train[i*599]
    plt.imshow(x_selected.squeeze(), cmap="gray")
    plt.title(list(y_train[i*599]).index(0))
    plt.axis('off')
plt.show()

### 1.5、构建网络模型

卷积神经网络（Convolutional Neural Network,CNN）是一种常见的深度学习架构，其初期主要是用来解决图像识别的问题，但早期由于缺乏训练数据和计算能力，要在不产生过拟合的情况下训练高性能卷积神经网络是很困难的。近年来GPU的发展，使得卷积神经网络研究涌现并取得一流结果，其表现的应用已经不仅仅应用在图像方面了，可运用在音频、自然语言处理等方面。

卷积神经网络受生物自然视觉认知机制启发而来，20世纪 90 年代，LeCun et al. 等人发表论文，确立了CNN的现代结构，后来又对其进行完善。他们设计了一种多层的人工神经网络，取名叫做**LeNet-5**，可以对手写数字做分类。2006年起，人们设计了很多方法，想要克服难以训练深度CNN的困难。其中，最著名的是 Krizhevsky et al.提出了一个经典的CNN 结构，并在图像识别任务上取得了重大突破。其方法的整体框架叫做**AlexNet(15.3%)**，与 LeNet-5 类似，但要更加深一些。AlexNet 取得成功后，研究人员又提出了其他的完善方法，其中最著名的要数**VGGNet(7.3%)**, **GoogleNet(6.7%)**和 **ResNet(3.57%)**这几种。从结构看，CNN 发展的一个方向就是层数变得更多，ILSVRC 2015 冠军 ResNet 是 AlexNet 的20 多倍，是 VGGNet 的8 倍多。通过增加深度，网络便能够利用增加的非线性得出目标函数的近似结构，同时得出更好的特性表征。但是，这样做同时也增加了网络的整体复杂程度，使网络变得难以优化，很容易过拟合,当然，研究人员们也提出了很多方法来解决这一问题。下图对比了目前各种卷积神经网络之间，复杂度和精度之间的关系。

![](images/pic2.png)

以LeNet为例子学习CNN。
![](images/LeNet1.png)

In [ ]:
# 构建模型
model = Sequential()
model.add(Conv2D(filters=6, kernel_size=(3,3), input_shape=(28, 28, 1), activation='relu')) #卷积层
model.add(MaxPooling2D(pool_size=(2,2))) #池化层
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu')) #卷积层
model.add(MaxPooling2D(pool_size=(2, 2)))#池化层
model.add(Flatten())# 拉平
model.add(Dense(120,activation='relu'))#全链接层
model.add(Dense(84,activation='relu'))#全链接层
model.add(Dense(10,activation='softmax'))

#### Flatten示意
![](images/flatten.png)

In [ ]:
model.summary()

In [ ]:
# 编译模型
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics=['accuracy'])

In [ ]:
# 训练模型
train_history=model.fit(x_train,y_train,batch_size=128,epochs=5,validation_data=(x_test,y_test))

In [ ]:
# 训练返回值
# 不同版本略有不同，有的为【acc】，有的为【accuracy】，后面画图自行修改
train_history.history

In [ ]:
# 绘图
import matplotlib.pyplot as plt
plt.plot(train_history.history['accuracy'])
plt.plot(train_history.history['val_accuracy'])
plt.title('Train History')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# 评估模型
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# 预测模型
prediction1 = model.predict(x_test)

In [ ]:
prediction1[:10]

In [ ]:
# 按类预测模型
prediction=model.predict_classes(x_test)

In [ ]:
prediction[:10]

In [ ]:
# 可视化部分结果
plt.figure(figsize=(12, 8))
for i in range(0, num_classes):
    plt.subplot(3, 4, i+1)
    x_selected = x_train[i*59:i*59+1]
    plt.imshow(x_selected.squeeze(), cmap="gray")
    titles = "real:" + str(list(y_train[i*59]).index(1))+",predict:"+str(model.predict(x_selected))
    print(titles)
    plt.axis('off')
plt.show()

In [ ]:
# 以混淆矩阵展现结果，对角线为预测正确的
import pandas as pd
(x_Train, y_Train), (x_Test, y_Test) = mnist.load_data()
pd.crosstab(y_Test,prediction,
            rownames=['label'],colnames=['predict'])

In [ ]:
df = pd.DataFrame({'label':y_Test, 'predict':prediction})

In [ ]:
# 取标签为6，预测为1的记录
df[(df.label==6)&(df.predict==1)]

### 总结

1. 学习了如何根据不同的模型数据要求，给原始数据图像增加维度
2. 学习了Conv2D卷积层和MaxPooling2D池化层的使用



<center>卷积神经网络与多层感知器进行数据预处理对比</center>  

||reshape|说明|  
|:-----:|:-----|:-----|
|多层感知器|image.reshape(60000,784)|多层感知器因为直接送进神经元处理，所以reshpe转换为60000项，每一项有784个数字|
|卷积神经网络|image.reshape(60000,28,28,1)|卷积神经网络则要求必须保持图像的维数，所以reshape转换60000项，每一项是$28\times 28\times 1$的图像|

# Any Questions?